In [2]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import torch
import torchaudio
import torchaudio.functional as audioF

import numpy as np

import os
import glob
import random

# Utility

In [ ]:
import pystoi
import pesq


def combine_audio(speech: torch.Tensor, noise: torch.Tensor, snr: torch.Tensor):
    return audioF.add_noise(speech, noise, snr)

def calc_pesq(speech: np.ndarray, processed: np.ndarray) -> float:
    return pesq.pesq(fs=16000,ref=speech,deg=processed)

def calc_stoi(speech: np.ndarray, processed: np.ndarray) -> float:
    return pystoi.stoi(x=speech,y=processed,fs_sig=16000)

In [ ]:
from torch.utils.data import Dataset
class AudioDataset(Dataset):
    def __init__(self, dir, root_dir=None):
        self.root_dir = root_dir
        if root_dir==None:
            self.root_dir = os.getcwd()+"\\data"
        self.data = glob.glob(dir,root_dir=root_dir)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        wave, _ = torchaudio.load(self.root_dir + "\\" + self.data[idx],normalize=False, format="wav")
        return wave

class AudioCombiner():
    def __init__(self, speech_ds: AudioDataset, noise_ds: AudioDataset):
        self.idx = 0
        self.speech_ds = speech_ds
        self.noise_ds = noise_ds

    def next(self, snr):
        self.idx += 1
        if self.idx >= len(self.speech_ds):
            raise IndexError(f"Index {self.idx} out of bounds of speech dataset ({len(self.speech_ds)}).")
        speech = self.speech_ds.__getitem__(self.idx)
        noise = self.noise_ds.__getitem__(random.randint(0,len(self.noise_ds)-1))
        return audioF.add_noise(speech, noise, torch.tensor(snr))

    def set_index(self, idx):
        self.idx = idx

        
    
speech_train_dataset = AudioDataset("speech\\train")
noise_train_dataset = AudioDataset("noise\\train")

# Models

In [ ]:
import tqdm

## SEGAN

In [ ]:
def segan_train()

## WaveCRN